In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as et
from io import BytesIO
from typing import List, Dict, Optional
from matplotlib import pyplot as plt
import numpy as np
from functools import cached_property

# Классы и методы

In [13]:
class FedStatIndicator:
    def __init__(self, indicator_id):
        self.id = indicator_id
        self._raw_data = None

    @cached_property
    def _filters_raw(self):
        """Получает сырые данные фильтров"""
        html = requests.get(f'https://www.fedstat.ru/indicator/{self.id}')
        if html.status_code == 200:
            soup = BeautifulSoup(html.text, "lxml")
            script = soup.find_all("script")[11].text
            pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
            match = re.search(pattern, script, re.DOTALL).group(0)
            filters = "{" + match + "}"
            filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
            filters = filters.replace("'", '"')
            filters_raw = json.loads(filters)['filters']
            return filters_raw
        else:
            raise requests.RequestException(f"Ошибка HTTP-запроса")

    @cached_property
    def filter_codes(self):
        """
        Возвращает код и название показателя, доступного для фильтрации
        """
        
        filter_codes = {key : self._filters_raw[key]['title'] for key in list(self._filters_raw.keys())[1:]}
        return filter_codes

    @cached_property
    def indicator_title(self):
        """
        Возвращает название индикатора
        """
        return list(self._filters_raw.get("0").get("values").values())[0]["title"]

    def get_filter_values(self):
        """
        Возвращает названия и коды доступных значений для выбранных фильтров
        """
        filters = self._filters_raw
        filter_codes = self.filter_codes
        
        categories =[]
        for key in filter_codes.keys():
            categories.append({
                key: list(filters[key]['values'].keys())
            })
        filter_ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
        return filter_ids
        
    @cached_property     
    def filter_categories(self):
        result_dict = {}
        for category in self.filter_codes.keys():
            values_data = self._filters_raw[category]['values']
            result_dict[category] = {f"{category}_{k}": v['title'] for k, v in values_data.items()}
        return result_dict

    def load_raw_indicator(self, data_type: str = "excel", filter_ids: List["str"] =  None):
        """
       Загружает данные индикатора через API

       :param data_type: тип возвращаемых данных
       :param filter_ids: список с кодами выбранных показателей
       :return: DataFrame с данными
        """

        if self._raw_data is not None:
            return self._raw_data

        if filter_ids is None:
            filter_ids = self.get_filter_values()

        data = {
                "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
                "columnObjectIds": ["3", "33560"],
                "selectedFilterIds": [
                filter_ids
                ]
            }
        params = {
            "format" : data_type,
            "id" : self.id  
        }
        try: 
            response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
            response.raise_for_status()
            if "excel" in response.headers.get("Content-Type"):
                self._raw_data = pd.read_excel(BytesIO(response.content), engine = "xlrd", header = 2) 
                return self._raw_data
            else:
                raise ValueError(
                    f"Ответ не содержит {data_type} файла"
                )
        except requests.RequestException as e:
            raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

    @staticmethod
    def _get_min_age(age_str):
        numbers = re.findall(r'\d+', age_str)
        return int(numbers[0]) if numbers else None

    @staticmethod
    def _get_max_age(age_str):
        numbers = re.findall(r"\d+", age_str)
        return int(numbers[-1]) if numbers else None

    @staticmethod   
    def _categorize_age(age_str):
        numbers = re.findall(r'\d+', age_str)
        words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
        if len(numbers) == 1 and len(words) <=3:
            return 1
        elif len(numbers) == 2 and len(words) <= 1:
            if int(numbers[-1]) - int(numbers[0]) == 4:
                return 2
            else:
                return 3
        else:
            return 4
    
    def _preprocess_dataframe(self, df):
     
        df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
        df = df.drop(0, axis = 0).reset_index(drop = True)

        df.rename(columns = {
            'Unnamed: 1' : "region",
            "Unnamed: 2" : "age",
            "Unnamed: 3" : "settlement"
            }, inplace = True)

        df.region = df.region.str.strip()
        years = [col for col in df.columns if col.isdigit()]
        df[years] = df[years].astype("Int64")
         
        df['age'] = df['age'].str.replace(r'\s*(лет|года|год)$', '', regex = True).str.strip()
        df = df.drop_duplicates(subset = ['region', 'age', 'settlement'], keep = 'last')
        
        return df

    def _remove_districts(self, df):

        """
        Удаляет федеральные округа из данных.
        :param df: DataFrame, предварительно обработанный
        :return: DataFrame без округов
        """
        
        rm_districts = '|'.join([
            r'Южный федеральный',
            r'Сибирский федеральный',
            r'Северо-Кавказский федеральный',
            r'Дальневосточный федеральный'
        ])

        districts_mask = df['region'].str.contains(rm_districts, case=False)
        df = df[~districts_mask].copy().reset_index(drop = True)
        return df

    def _change_districts(self, df):
        
        """
        Группирует регионы по федеральным округам.
        :param df: DataFrame после удаления округов
        :return: DataFrame с добавленными строками по округам
        """
        districts_config = {
            'Дальневосточный федеральный округ' : '|'.join([
                r'Бурятия',
                r'Забайкал',
                r'Саха',
                r'Якутия',
                r'Камчат',
                r'Приморск',
                r'Хабаровский',
                r'Амурская',
                r'Магаданская',
                r'Сахалинская',
                r'Еврейская автономная область',
                r'Чукотский автономный округ']),
            'Сибирский федеральный округ' : '|'.join([
                r'Алтай',
                r'Тыва',
                r'Хакасия',
                r'Красноярский',
                r'Иркутская',
                r'Кемеровская',
                r'Кузбасс',
                r'Новосибирская',
                r'Омская',
                r'Томская']),
            'Южный федеральный округ': '|'.join([
                r'Адыгея', 
                r'Калмыкия',
                r'Краснодарский',
                r'Астраханская',
                r'Волгоградская',
                r'Ростовская',
                r'Крым',
                r'Севастополь'
                ]),
            'Северо-Кавказский федеральный округ' :  '|'.join([
                r'Дагестан',
                r'Ингушетия',
                r'Кабардин',
                r'Балкар',
                r'Карачаев',
                r'Черкес',
                r'Осетия',
                r'Алания',
                r'Чечен',
                r'Ставрополь'
            ])   
        }
        aggregated_dfs = []
        for district_name, regions in districts_config.items():
            mask = df['region'].str.contains(regions, case = False, na = False)
            district_df = df[mask].copy()
            if not district_df.empty:
                agg_df = district_df.groupby(
                    ['age', 'settlement'],
                    as_index = False
                ).sum(numeric_only = True)
                agg_df.insert(0, 'region', district_name)
                aggregated_dfs.append(agg_df)
        df = pd.concat([df] + aggregated_dfs, ignore_index = True)
        return df   

    def _add_mid_year_values(self, df):
        
        if df is None:
            df = self._change_districts()
        
        df['min_age']  = df['age'].apply(self._get_min_age)
        df['max_age'] = df['age'].apply(self._get_max_age)
        df['age_category'] = df['age'].apply(self._categorize_age)

        

        min_year = min([int(col) for col in df.columns if col.isdigit()])
        df_mid = pd.DataFrame()

        for col in df.columns:
            if col.isdigit():
                if int(col) > min_year:
                    df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
                df_mid[f"{col}end"] = df[col]
            else:
                df_mid[col] = df[col]        
        
        df_mid = df_mid.groupby('region', sort = False).apply(
                lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
                ).reset_index(drop = True)
        return df_mid

    def get_processed_data(self, data_type: str = "excel", filter_ids: List["str"] =  None ):
        
        """
        Загружает, очищает и агрегирует данные Росстата для дальнейшего анализа.

        Последовательность обработки данных:
        1. Загружает исходные данные индикатора через API (метод `load_raw_indicator`).
        2. Выполняет предварительную очистку таблицы:
        - Переименовывает столбцы,
        - Удаляет лишние строки и столбцы,
        - Приводит числовые столбцы к типу Int64.
        3. Удаляет из данных строки с федеральными округами (оставляет только регионы).
        4. Агрегирует данные по федеральным округам:
        - Объединяет данные по регионам в укрупнённые федеральные округа.
        5. Добавляет дополнительные столбцы со значениями за середину года (среднее между двумя годами).

        :param data_type: Формат загружаемых данных с сайта (по умолчанию "excel").
        :param filter_ids: Список кодов фильтров для отбора данных (если не указан, загружаются все доступные).
        :return: pandas.DataFrame с итоговыми очищенными и агрегированными данными.
        """

        raw_df = self.load_raw_indicator(
            data_type = data_type,
            filter_ids = filter_ids
        )
        df = self._preprocess_dataframe(raw_df)
        df = self._remove_districts(df)
        df = self._change_districts(df)
        df = self._add_mid_year_values(df)
        return df

In [11]:
women_population = FedStatIndicator(indicator_id = 33459)

# Женское население

In [12]:
df_women = women_population.get_processed_data()

C:\Users\User\AppData\Local\Temp\ipykernel_7284\1150023640.py:252: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


In [14]:
df_women.shape

(42655, 77)

In [15]:
df_women['age'] = df_women['age'].str.replace(r'\s*(лет|года|год)$', '', regex = True).str.strip()
df_women = df_women.drop_duplicates(subset = ['region', 'age', 'settlement'], keep = 'last')

In [16]:
df_women.shape

(42655, 77)

In [91]:
df_women.to_excel("women_population.xlsx", engine = "openpyxl", index = False)

## SDMX data

In [ ]:
#All values
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": ids
}
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

In [7]:
# Adding  regions
regions = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="57831" for val in v]

In [8]:
#Adding years
years = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="3" for val in v]

In [9]:
#Adding settlement type
settlement_type = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58274" for val in v]
settlement_type

['58274_1744150', '58274_1750789', '58274_1750788']

In [10]:
#Adding age categories
age =  [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58335" for val in v]

In [12]:
#collect regional data
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            regions,
            "58274_1744150",
            "58335_1709566"
        ]
    }
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'text/xml', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 10:39:48 GMT', 'Set-Cookie': 'JSESSIONID=20E85FCD89FE3297FE03E7FAF9608248; Path=/; Secure; HttpOnly, session-cookie=18517ad4dc4c6a7edcd5a505204218c4d2506ea5bd73d1c7df057e3ecf38e193bc79ec9cdd66ebf6efc4768835a5ee4b; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xml", 'vary': 'accept-encoding', 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [13]:
# Save the data in a file
with open("regions.xml", "wb") as file:
    file.write(response.content)

In [41]:
#Transform sdmx file into pandas dataframe
def read_sdmx(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    namespaces = {
        "generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
        "message": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
        "structure": 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    } 

    dataset = root.find(".//message:DataSet", namespaces)
    data = []
    for series in dataset.findall(".//generic:Series", namespaces):
        observations = series.findall(".//generic:Obs", namespaces)
        pair = {}
        for obs in observations:
            time = obs.find(".//generic:Time", namespaces).text
            value = obs.find(".//generic:ObsValue", namespaces).attrib['value']
            region = obs.find(".//")
            data.append({
                "year" : time,
                "population" : value
                })
    return pd.DataFrame(data)

data = read_sdmx("regions.xml")
data

,year,population
0,1990,78549601
1,1991,78817695
2,1992,78914788
3,1993,78899303
4,1994,78770458
...,...,...
3502,1996,3833075
3503,1999,3700204
3504,2000,3620686
3505,2001,3620686


## Excel data

In [40]:
#Let's get the data in excel
def load_indicator(id, data_type, regions, years, settlement_type, age):
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                regions,
                "0_33459",
                years,
                "30611_950458",
                "33560_1540248",
                settlement_type,
                age
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
    return response
# print(response.status_code, response.headers)

In [7]:
#Load the data
df = pd.read_excel("all_data.xls", header = 2)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1990,1991,1992,1993,1994,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
1,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,все население,человек,78549601,78817695,78914788,78899303,78770458,...,78958558,79137859,79207619,79202094,79219390,78952485,78601633,78354712,78233436,78154709
2,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,городское население,человек,57983496,58303864,58267844,57878461,57688561,...,59279662,59512332,59660317,59755068,59858643,59691444,59487917,59352665,59312967,59399259
3,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,сельское население,человек,20566105,20513831,20646944,21020842,21081897,...,19678896,19625527,19547302,19447026,19360747,19261041,19113716,19002047,18920469,18755450
4,Численность постоянного населения - женщин по ...,Российская Федерация,0,все население,человек,1035959,957048,868672,770771,675810,...,942238,910777,814676,771071,714593,688397,673494,632575,611254,593327


In [8]:
#Remove unnecassary columns
df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
df = df.drop(0, axis = 0).reset_index(drop = True)

In [9]:
# Rename
df.rename(columns = {
    'Unnamed: 1' : "region",
    "Unnamed: 2" : "age",
    "Unnamed: 3" : "settlement"
    }, inplace = True)

In [10]:
df.region = df.region.str.strip()

In [11]:
#Remove unnecessary districts
rm_districts = '|'.join([
    r'Южный федеральный',
    r'Сибирский федеральный',
    r'Северо-Кавказский федеральный',
    r'Дальневосточный федеральный'
])
districts_mask = df['region'].str.contains(rm_districts, case=False)
df= df[~districts_mask].reset_index(drop = True)

In [12]:
years = [col for col in df.columns if col.isdigit()]
df[years] = df[years].astype("Int64")

In [13]:
def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

df['min_age']  = df['age'].apply(get_min_age)
df['max_age'] = df['age'].apply(get_max_age)
df['age_category'] = df['age'].apply(categorize_age)

In [14]:
#Add mid-year value
min_year = min([int(col) for col in df.columns if col.isdigit()])
df_mid = pd.DataFrame()

for col in df.columns:
    if col.isdigit():
        if int(col) > min_year:
            df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
        df_mid[f"{col}end"] = df[col]
    else:
        df_mid[col] = df[col]
df_mid.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,Всего,все население,78549601,78683648.0,78817695,78866241.5,78914788,78907045.5,78899303,...,78601633,78478172.5,78354712,78294074.0,78233436,78194072.5,78154709,NaN,NaN,4
1,Российская Федерация,Всего,городское население,57983496,58143680.0,58303864,58285854.0,58267844,58073152.5,57878461,...,59487917,59420291.0,59352665,59332816.0,59312967,59356113.0,59399259,NaN,NaN,4
2,Российская Федерация,Всего,сельское население,20566105,20539968.0,20513831,20580387.5,20646944,20833893.0,21020842,...,19113716,19057881.5,19002047,18961258.0,18920469,18837959.5,18755450,NaN,NaN,4
3,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
4,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1


In [15]:
#Fix federal districts

dfo_regions = [
    'Республика Бурятия',
    'Забайкальский край',
    'Республика Саха (Якутия)',
    'Камчатский край',
    'Приморский край',
    'Хабаровский край',
    'Амурская область',
    'Магаданская область',
    'Сахалинская область',
    'Еврейская автономная область',
    'Чукотский автономный округ'
]
dfo_mask = df_mid['region'].isin(dfo_regions)
dfo = df_mid[dfo_mask]
dfo_agg = dfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
dfo_agg.insert(0, 'region', 'Дальневосточный федеральный округ')

In [16]:
sfo_regions = ['Республика Алтай',
    'Республика Тыва',
    'Республика Хакасия',
    'Алтайский край',
    'Красноярский край',
    'Иркутская область',
    'Кемеровская область - Кузбасс',
    'Новосибирская область',
    'Омская область',
    'Томская область']
sfo_mask = df_mid['region'].isin(sfo_regions)
sfo = df_mid[sfo_mask]
sfo_agg = sfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
sfo_agg.insert(0, 'region', 'Сибирский федеральный округ')

In [17]:
ufo_pattern = '|'.join([
    r'Адыгея', 
    r'Калмыкия',
    r'Краснодарский',
    r'Астраханская',
    r'Волгоградская',
    r'Ростовская',
    r'Крым',
    r'Севастополь'
])

skfo_pattern = '|'.join([
    r'Дагестан',
    r'Ингушетия',
    r'Кабардин',
    r'Балкар',
    r'Карачаев',
    r'Черкес',
    r'Осетия',
    r'Алания',
    r'Чечен',
    r'Ставрополь'
])

ufo_mask = df_mid['region'].str.contains(ufo_pattern, case = False, na = False)
ufo = df_mid[ufo_mask]
ufo_agg = ufo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
ufo_agg.insert(0, 'region', 'Южный федеральный округ')

skfo_mask = df_mid['region'].str.contains(skfo_pattern, case = False, na = False)
skfo = df_mid[skfo_mask]
skfo_agg = skfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
skfo_agg.insert(0, 'region', 'Северо-Кавказский федеральный округ')

df_mid = pd.concat([df_mid, ufo_agg, skfo_agg, sfo_agg], ignore_index = True)


In [19]:
df_mid = df_mid.groupby('region', sort = False).apply(
    lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
).reset_index(drop = True)
df_mid.head()

C:\Users\User\AppData\Local\Temp\ipykernel_5108\2115352455.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
1,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1
2,Российская Федерация,0,сельское население,304913,297807.0,290701,282420.5,274140,264623.5,255107,...,167268,162402.0,157536,155239.5,152943,149004.5,145066,0.0,0.0,1
3,Российская Федерация,1 год,все население,1132245,1082223.5,1032202,996674.5,961147,915514.0,869881,...,689192,681807.5,674423,653844.5,633266,623035.5,612805,1.0,1.0,1
4,Российская Федерация,1 год,городское население,799394,763825.0,728256,697132.0,666008,627874.0,589740,...,519345,513412.5,507480,491841.0,476202,468292.5,460383,1.0,1.0,1


In [ ]:
#Highlight "total" rows with red color
def highlight_total(row, color = 'red'):
    return [f"background-color: {color}" for _ in row]
df_styled = df_mid.style.apply(
    lambda row : highlight_total(row) if row['age'] == 'Всего' else ['']*len(row),
    axis = 1
)
df_styled

In [ ]:
#Save in a csv file
df_mid.drop(columns = ['age_category', 'min_age', 'max_age'], inplace = True)
df_mid.to_excel("sorted_data.xlsx", index = False)
df_styled.to_excel("sorted_data_highlighted.xlsx", index = False)

# Мужское население

In [75]:
men_population = FedStatIndicator(indicator_id = 31548)

In [76]:
df_men = men_population.get_processed_data()

C:\Users\User\AppData\Local\Temp\ipykernel_7284\1150023640.py:252: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


In [77]:
df_men.shape

(44292, 77)

In [78]:
df_men['age'] = df_men['age'].str.replace(r'\s*(лет|года|год)$', '', regex = True).str.strip()
df_men = df_men.drop_duplicates(subset = ['region', 'age', 'settlement'], keep = 'last')

In [79]:
df_men.shape

(42657, 77)

In [80]:
columns = ['region', 'age', 'settlement']
diff = df_men[columns].merge(df_women[columns], how='outer', indicator=True)

In [81]:
diff_values = diff[diff['_merge'] == 'left_only']

mask = df_men[['region', 'age', 'settlement']].apply(tuple, axis=1).isin(
    diff_values[['region', 'age', 'settlement']].apply(tuple, axis=1)
)
df_men = df_men[~mask]

In [82]:
df_men.reset_index(drop = True, inplace = True)

In [83]:
df_men.shape[0] == df_women.shape[0]

True

### В мужском населениии за 2021 год есть пропуски. Их надо восстановить
1) Через соотношение численности мужчин и женщин, предполагая, что оно не сильно меняется
2) Линенейная регрессия

In [38]:
df_rel = df_men.select_dtypes(include = ['object']).copy()
year_cols = [col for col in df_men.columns if "end" in col]

for col in year_cols:
    df_rel[col] = df_women[col] / df_men[col]
df_rel['age_category'] = df_men['age_category']
df_rel.tail()

,region,age,settlement,1990end,1991end,1992end,1993end,1994end,1995end,1996end,...,2017end,2018end,2019end,2020end,2021end,2022end,2023end,2024end,2025end,age_category
42650,Северо-Кавказский федеральный округ,Моложе трудоспособного,городское население,0.0,0.0,0.964584,0.0,0.961477,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
42651,Северо-Кавказский федеральный округ,Моложе трудоспособного,сельское население,0.0,0.0,0.983439,0.0,0.981343,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
42652,Северо-Кавказский федеральный округ,Старше трудоспособного,все население,2.452334,2.372658,2.387259,2.368418,2.393801,2.396854,2.356667,...,2.130537,2.095782,2.04951,2.070781,2.02579,2.062531,1.98393,2.010198,1.964749,4
42653,Северо-Кавказский федеральный округ,Старше трудоспособного,городское население,2.416379,2.355731,2.369155,2.379809,2.428731,2.461577,2.436751,...,2.160789,2.130549,2.080941,2.099327,2.058422,2.092114,2.016542,2.040902,2.003899,4
42654,Северо-Кавказский федеральный округ,Старше трудоспособного,сельское население,2.492393,2.391736,2.407287,2.356472,2.358519,2.334794,2.280732,...,2.097399,2.058176,2.015702,2.039965,1.991199,2.031288,1.949706,1.977652,1.923068,4


In [39]:
df_rel.to_excel('relations.xlsx', index = False)

In [40]:
df_men.to_excel("men_population.xlsx")

In [133]:
df_women_1 = df_women[df_women['age_category'] == 1].reset_index(drop = True)
df_men_1 = df_men[df_men['age_category'] == 1].reset_index(drop = True)

df_rel = df_men_1.select_dtypes(include = ['object']).copy()
year_cols = [col for col in df_men.columns if "end" in col]

for col in year_cols:
    df_rel[col] = df_women_1[col] / df_men_1[col]
df_rel['age_category'] = df_men_1['age_category']

In [61]:
import numpy as np
df_rel = df_men.select_dtypes(include = ['object']).copy()
year_cols = [col for col in df_men.columns if "end" in col]

for col in year_cols:
    df_rel[col] = df_men[col] / df_women[col]
df_rel['age_category'] = df_men['age_category']
df_rel.head()

,region,age,settlement,1990end,1991end,1992end,1993end,1994end,1995end,1996end,...,2017end,2018end,2019end,2020end,2021end,2022end,2023end,2024end,2025end,age_category
0,Российская Федерация,0,все население,1.049821,1.052715,1.049141,1.05614,1.055596,1.051501,1.04927,...,1.058819,1.060352,1.065792,1.060427,<NA>,1.060805,1.056359,1.063429,1.062979,1
1,Российская Федерация,0,городское население,1.049476,1.055416,1.052396,1.056651,1.056029,1.054311,1.04939,...,1.058538,1.060809,1.066872,1.058238,<NA>,1.060834,1.057109,1.066579,1.06537,1
2,Российская Федерация,0,сельское население,1.050647,1.046525,1.042081,1.055106,1.054727,1.045703,1.049018,...,1.059672,1.058985,1.062565,1.067104,<NA>,1.060717,1.054096,1.053987,1.055588,1
3,Российская Федерация,1,все население,1.040973,1.049245,1.05092,1.048854,1.054741,1.05576,1.050464,...,1.056971,1.058746,1.060201,1.065722,<NA>,1.061118,1.060692,1.056286,1.063138,1
4,Российская Федерация,1,городское население,1.042234,1.049682,1.054201,1.05184,1.056102,1.056781,1.053332,...,1.058169,1.058181,1.060277,1.066435,<NA>,1.061572,1.060954,1.056699,1.066128,1


In [90]:
mean_2021 = (df_rel['2020end'] + df_rel['2022end']) / 2
df_rel["2021end"] = df_rel["2021end"].fillna(mean_2021)
df_rel.head()

,region,age,settlement,1990end,1991end,1992end,1993end,1994end,1995end,1996end,1997end,1998end,1999end,2000end,2001end,2002end,2003end,2004end,2005end,2006end,2007end,2008end,2009end,2010end,2011end,2012end,2013end,2014end,2015end,2016end,2017end,2018end,2019end,2020end,2021end,2022end,2023end,2024end,2025end,age_category
0,Российская Федерация,0,все население,1.049821,1.052715,1.049141,1.05614,1.055596,1.051501,1.04927,1.053619,1.05346,1.055185,1.05746,1.058288,1.054519,1.050808,1.058618,1.055247,1.053458,1.053978,1.05742,1.052959,1.054801,1.05368,1.058794,1.0613,1.058753,1.062376,1.057156,1.058819,1.060352,1.065792,1.060427,1.060616,1.060805,1.056359,1.063429,1.062979,1
1,Российская Федерация,0,городское население,1.049476,1.055416,1.052396,1.056651,1.056029,1.054311,1.04939,1.058387,1.058434,1.055716,1.060916,1.060415,1.055777,1.055027,1.060766,1.058451,1.055295,1.056745,1.058851,1.054486,1.055587,1.055496,1.060839,1.064437,1.058699,1.063542,1.058803,1.058538,1.060809,1.066872,1.058238,1.059536,1.060834,1.057109,1.066579,1.06537,1
2,Российская Федерация,0,сельское население,1.050647,1.046525,1.042081,1.055106,1.054727,1.045703,1.049018,1.043514,1.042811,1.05403,1.049979,1.053514,1.051625,1.040844,1.053346,1.047467,1.048974,1.047367,1.054134,1.049474,1.052962,1.049282,1.05392,1.053593,1.058887,1.059405,1.052375,1.059672,1.058985,1.062565,1.067104,1.063911,1.060717,1.054096,1.053987,1.055588,1
3,Российская Федерация,1,все население,1.040973,1.049245,1.05092,1.048854,1.054741,1.05576,1.050464,1.047316,1.051733,1.051521,1.053325,1.055183,1.052562,1.048711,1.049991,1.056389,1.052911,1.04877,1.052627,1.055634,1.050907,1.053,1.054369,1.058616,1.061156,1.058291,1.062233,1.056971,1.058746,1.060201,1.065722,1.06342,1.061118,1.060692,1.056286,1.063138,1
4,Российская Федерация,1,городское население,1.042234,1.049682,1.054201,1.05184,1.056102,1.056781,1.053332,1.047775,1.057892,1.055738,1.054441,1.058342,1.05407,1.048787,1.053407,1.058042,1.055691,1.050372,1.054807,1.056614,1.052389,1.055224,1.056356,1.06013,1.063944,1.058331,1.062996,1.058169,1.058181,1.060277,1.066435,1.064003,1.061572,1.060954,1.056699,1.066128,1


In [73]:
men_2021 = df_women["2021end"] * df_rel["2021end"]
men_2021 = men_2021.replace([np.inf, -np.inf], np.nan)
# df_men["2021end"] = df_men["2021end"].astype("float")
df_men["2021end"] = df_men["2021end"].fillna(men_2021)
df_men.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,1994mid,1994end,1995mid,1995end,1996mid,1996end,1997mid,1997end,1998mid,1998end,1999mid,1999end,2000mid,2000end,2001mid,2001end,2002mid,2002end,2003mid,2003end,2004mid,2004end,2005mid,2005end,2006mid,2006end,2007mid,2007end,2008mid,2008end,2009mid,2009end,2010mid,2010end,2011mid,2011end,2012mid,2012end,2013mid,2013end,2014mid,2014end,2015mid,2015end,2016mid,2016end,2017mid,2017end,2018mid,2018end,2019mid,2019end,2020mid,2020end,2021mid,2021end,2022mid,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,1087571,1047535.0,1007499,959429.0,911359,862700.5,814042,763712.0,713382,724793.0,736204,719871.5,703539,689218.5,674898,662641.5,650385,656117.5,661850,644816.0,627782,639515.0,651248,657313.5,663379,683304.5,703230,727264.5,751299,756893.0,762487,752798.0,743109,747225.0,751341,785603.5,819866,844951.0,870036,878167.0,886298,869388.0,852478,883036.0,913594,941094.0,968594,965560.0,962526,974999.5,987473,991783.0,996093,980220.5,964348,914095.5,863843,842822.0,821801,789787.5,757774,<NA>,730125.112501,<NA>,714446,691336.0,668226,659125.5,650025,640359.5,630694,0.0,0.0,1
1,Российская Федерация,0,городское население,767215,735244.0,703273,664478.0,625683,585280.0,544877,510583.5,476290,486742.0,497194,487296.5,477399,469013.0,460627,453018.0,445409,449484.0,453559,442172.0,430785,441087.5,451390,457191.5,462993,479496.0,495999,515433.0,534867,538307.0,541747,534883.0,528019,529552.5,531086,551445.0,571804,588829.0,605854,613685.5,621517,612963.5,604410,624612.5,644815,667616.0,690417,689533.0,688649,699323.5,709998,726018.0,742038,733734.0,725430,686430.5,647431,631875.5,616320,592890.0,569460,<NA>,548632.061848,<NA>,537022,519595.0,502168,495496.5,488825,483194.5,477564,0.0,0.0,1
2,Российская Федерация,0,сельское население,320356,312291.0,304226,294951.0,285676,277420.5,269165,253128.5,237092,238051.0,239010,232575.0,226140,220205.5,214271,209623.5,204976,206633.5,208291,202644.0,196997,198427.5,199858,200122.0,200386,203808.5,207231,211831.5,216432,218586.0,220740,217915.0,215090,217672.5,220255,234158.5,248062,256122.0,264182,264481.5,264781,256424.5,248068,258423.5,268779,273478.0,278177,276027.0,273877,275676.0,277475,265765.0,254055,246486.5,238918,227665.0,216412,210946.5,205481,196897.5,188314,<NA>,181495.689312,<NA>,177424,171741.0,166058,163629.0,161200,157165.0,153130,0.0,0.0,1
3,Российская Федерация,1,все население,1178636,1130834.5,1083033,1046561.0,1010089,961233.5,912378,864359.0,816340,767993.0,719646,727797.5,735949,719760.0,703571,688723.5,673876,661330.5,648785,654482.0,660179,644372.0,628565,638528.0,648491,651759.0,655027,680658.5,706290,726952.0,747614,752211.5,756809,749755.5,742702,745704.0,748706,782394.0,816082,839854.5,863627,862562.0,861497,858251.5,855006,884034.0,913062,940330.0,967598,971953.5,976309,981917.5,987526,991600.0,995674,979972.5,964271,914081.5,863892,843042.5,822193,<NA>,759989.127902,<NA>,731314,723334.5,715355,692132.5,668910,660203.0,651496,1.0,1.0,1
4,Российская Федерация,1,городское население,833156,798796.5,764437,733271.5,702106,661209.0,620312,582792.0,545272,513086.5,480901,489286.5,497672,487690.5,477709,468743.0,459777,451848.5,443920,447245.0,450570,440620.5,430671,438918.5,447166,451691.5,456217,477639.5,499062,513642.0,528222,531935.5,535649,531583.5,527518,528362.5,529207,548772.0,568337,584596.5,600856,604285.0,607714,607111.5,606509,625709.5,644910,667692.0,690474,693845.0,697216,704150.5,711085,727073.0,743061,734921.0,726781,687832.0,648883,633360.0,617837,<NA>,573113.731278,<NA>,551322,544867.5,538413,520807.5,503202,497014.5,490827,1.0,1.0,1


In [74]:
def remove_differences(df1, df2):
    df_1, df_2 = df1.copy(), df2.copy()
    df1_size, df2_size = df1.shape[0], df2.shape[0]
    if df1_size == df2_size:
        return df_1, df_2

    columns = ['region', 'age', 'settlement']
    bigger_df, smaller_df = (df_1, df_2) if df1_size > df2_size else (df_2, df_1) 

    diff = bigger_df[columns].merge(smaller_df[columns], how='outer', indicator=True)
    diff_values = diff[diff['_merge'] == 'left_only']

    mask = bigger_df[['region', 'age', 'settlement']].apply(tuple, axis=1).isin(
        diff_values[['region', 'age', 'settlement']].apply(tuple, axis=1)
    )
    bigger_df = bigger_df[~mask]

    if df1_size > df2_size:
        df_1, df_2 = bigger_df, smaller_df
    else:
        df_2, df_1 = bigger_df, smaller_df

    if len(df_1) != len(df_2):
        raise ValueError(
            f"Размеры не совпали после обработки: df1={len(df_1)}, df2={len(df_2)}. "
            "Возможно, в данных есть дубликаты ключевых колонок."
        )
    return df_1, df_2


In [ ]:
def fill_missing_column(df_men, df_women):
    df_men = df_men.copy()
    df_men = df_men.reset_index(drop = True, inplace = True)
    df_rel = df_men.select_dtypes(include = ['object']).copy()
    year_cols = [col for col in df_men.columns if "end" in col]

    for col in year_cols:
        df_rel[col] = df_men[col] / df_women[col].replace(0, np.nan)
    df_rel['age_category'] = df_men['age_category']
    
    mean_2021 = (df_rel['2020end'] + df_rel['2022end']) / 2
    df_rel["2021end"] = df_rel["2021end"].fillna(mean_2021)
    men_2021 = df_women["2021end"] * df_rel["2021end"]
    men_2021 = men_2021.replace([np.inf, -np.inf], np.nan)
    df_men["2021end"] = df_men["2021end"].astype("float")
    df_men["2021end"] = df_men["2021end"].fillna(men_2021)

    df_men["2022mid"] = (df_men["2022end"] + df_men["2021end"]) / 2
    df_men["2021mid"] = (df_men["2021end"] + df_men["2020end"]) / 2
    return df_men
df_men_new = fill_missing_column(df_men, df_women)

In [115]:
def total_sum(df_men, df_women):
    df_men = df_men.copy()
    df_women = df_women.copy()
    df_men = df_men.reset_index(drop = True)
    df_women = df_women.reset_index(drop = True)

    df_all = pd.DataFrame()
    year_cols = [col for col in df_men.columns if "end" in col or "mid" in col]
    for col in df_men.columns:
        if col in year_cols:
            df_all[col] = df_men[col] + df_women[col]
        else:
            df_all[col] = df_men[col]
    return df_all
df_all = total_sum(df_men_new, df_women)
df_all.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,1994mid,1994end,1995mid,1995end,1996mid,1996end,1997mid,1997end,1998mid,1998end,1999mid,1999end,2000mid,2000end,2001mid,2001end,2002mid,2002end,2003mid,2003end,2004mid,2004end,2005mid,2005end,2006mid,2006end,2007mid,2007end,2008mid,2008end,2009mid,2009end,2010mid,2010end,2011mid,2011end,2012mid,2012end,2013mid,2013end,2014mid,2014end,2015mid,2015end,2016mid,2016end,2017mid,2017end,2018mid,2018end,2019mid,2019end,2020mid,2020end,2021mid,2021end,2022mid,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,0,все население,2123530,2044038.5,1964547,1872289.0,1780031,1682422.0,1584813,1487002.5,1389192,1412771.0,1436350,1405196.0,1374042,1344746.0,1315450,1291607.5,1267765,1278425.5,1289086,1255269.0,1221452,1244039.5,1266627,1279544.0,1292461,1332459.5,1372458,1416727.5,1460997,1473025.5,1485054,1466781.5,1448509,1456356.0,1464203,1529707.5,1595212,1645762.5,1696313,1711431.0,1726549,1694037.5,1661526,1718991.5,1776457,1828850.0,1881243,1876441.0,1871639,1894303.5,1916968,1927649.5,1938331,1906728.0,1875125,1776822.0,1678519,1635695.5,1592872,1532619.5,1472367,1445444.556251,1418522.112501,1403231.056251,1387940,1344370.5,1300801,1281040.0,1261279,1242650.0,1224021,0.0,0.0,1
1,Российская Федерация,0,городское население,1498261,1433940.5,1369620,1294917.5,1220215,1140378.0,1060541,993925.5,927310,948043.0,968776,950552.5,932329,914086.0,895843,881035.5,866228,874704.5,883181,860008.0,836835,856949.0,877063,889294.5,901526,933827.0,966128,1002611.0,1039094,1046335.5,1053577,1040974.0,1028371,1031012.5,1033654,1072740.5,1111827,1146115.0,1180403,1195354.0,1210305,1193673.0,1177041,1214845.5,1252650,1295844.5,1339039,1339077.5,1339116,1358346.5,1377577,1410221.0,1442865,1426804.0,1410743,1334246.0,1257749,1225879.0,1194009,1150795.0,1107581,1087008.530924,1066436.061848,1054842.030924,1043248,1010227.5,977207,962171.5,947136,936480.5,925825,0.0,0.0,1
2,Российская Федерация,0,сельское население,625269,610098.0,594927,577371.5,559816,542044.0,524272,493077.0,461882,464728.0,467574,454643.5,441713,430660.0,419607,410572.0,401537,403721.0,405905,395261.0,384617,387090.5,389564,390249.5,390935,398632.5,406330,414116.5,421903,426690.0,431477,425807.5,420138,425343.5,430549,456967.0,483385,499647.5,515910,516077.0,516244,500364.5,484485,504146.0,523807,533005.5,542204,537363.5,532523,535957.0,539391,517428.5,495466,479924.0,464382,442576.0,420770,409816.5,398863,381824.5,364786,358437.344656,352088.689312,348390.344656,344692,334143.0,323594,318868.5,314143,306169.5,298196,0.0,0.0,1
3,Российская Федерация,1,все население,2310881,2213058.0,2115235,2043235.5,1971236,1876747.5,1782259,1686285.5,1590312,1495798.0,1401284,1418913.5,1436543,1405949.5,1375356,1344980.5,1314605,1290193.5,1265782,1276359.0,1286936,1255597.0,1224258,1244428.0,1264598,1272113.5,1279629,1329291.0,1378953,1417137.0,1455321,1465454.0,1475587,1463227.0,1450867,1455423.5,1459980,1524567.5,1589155,1637287.0,1685419,1682526.0,1679633,1672778.0,1665923,1720745.0,1775567,1827499.5,1879432,1889137.5,1898843,1908019.5,1917196,1927438.5,1937681,1906359.5,1875038,1776884.0,1678730,1636206.0,1593682,1534168.063951,1474654.127902,1447580.063951,1420506,1405142.0,1389778,1345977.0,1302176,1283238.5,1264301,1.0,1.0,1
4,Российская Федерация,1,городское население,1632550,1562621.5,1492693,1430403.5,1368114,1289083.0,1210052,1135815.0,1061578,998770.5,935963,953054.5,970146,951891.0,933636,914014.5,894393,879398.0,864403,871140.0,877877,857739.0,837601,854497.5,871394,881303.0,891212,932017.0,972822,1000144.5,1027467,1035254.0,1043041,1036389.5,1029738,1030327.5,1030917,1068569.5,1106222,1139011.5,1171801,1177712.5,1183624,1182142.5,1180661,1216951.0,1253241,1296345.5,1339450,1347727.0,1356004,1368016.5,1380029,1412652.0,1445275,1429438.5,1413602,1337239.5,1260877,1229031.0,1197185,1154468.865639,1111752.731278,1091209.865639,1070667,1058280.0,1045893,10126

# Опция ввода информации через чат

In [162]:
men_population.filter_categories["58335"].values()

dict_values(['Всего', '0', '0-17 лет', '0-4 года', '1 год', '10 лет', '100 и более лет', '10-14 лет', '11 лет', '12 лет', '13 лет', '14 лет', '15 лет', '15-19 лет', '16 лет', '16-29 лет', '17 лет', '18 лет', '19 лет', '2 года', '20 лет', '20-24', '20-24 лет', '20-29 лет', '20-39 лет', '21 год', '22 лет', '23 лет', '24 лет', '25 лет', '25-29', '25-29 лет', '26 лет', '27 лет', '28 лет', '29 лет', '3 года', '30 лет', '30-34', '30-34 лет', '31 лет', '32 лет', '33 лет', '34 лет', '35 лет', '35-39', '35-39 лет', '36 лет', '37 лет', '38 лет', '39 лет', '4 года', '40 лет', '40-44', '40-44 лет', '41 лет', '42 лет', '43 лет', '44 лет', '45 лет', '45-49 лет', '46 лет', '47 лет', '48 лет', '49 лет', '5 лет', '50 лет', '50-54 лет', '51 лет', '52 лет', '53 года', '54 лет', '55 лет', '55-59 лет', '56 лет', '57 лет', '58 лет', '59 лет', '5-9 лет', '6 лет', '60 лет', '60-64 лет', '61 лет', '62 года', '63 года', '64 года', '65 года', '65-69 лет', '66 года', '67 лет', '68 лет', '69 лет', '7 лет', '70 лет

In [169]:
system_prompt = """Role: Professional text reader
Task: Extract structured information from user's text input and fill JSON template
Rules:
1. If information is missing use all allowed values for this key defined in "Allowed options" section
2. Always validate extracted values against allowed options

Output format: 
{"region" : ["<регионы>"],
"age" : ["<возраст>"],
"settlement" : ["<тип населения>"],
"years" : ["<год>"],
"gender": ["<пол>"]}

Allowed options:
- regions: ['Российская Федерация', 'Центральный федеральный округ', 'Белгородская область', 'Брянская область', 'Владимирская область', 'Воронежская область', 'Ивановская область', 'Калужская область', 'Костромская область', 'Курская область', 'Липецкая область', 'Московская область', 'Орловская область', 'Рязанская область', 'Смоленская область', 'Тамбовская область', 'Тверская область', 'Тульская область', 'Ярославская область', 'Город Москва столица Российской Федерации город федерального значения', 'Северо-Западный федеральный округ', 'Республика Карелия', 'Республика Коми', 'Архангельская область', 'Ненецкий автономный округ (Архангельская область)', 'Архангельская область (кроме Ненецкого автономного округа)', 'Вологодская область', 'Калининградская область', 'Ленинградская область', 'Мурманская область', 'Новгородская область', 'Псковская область', 'Город Санкт-Петербург город федерального значения', 'Южный федеральный округ (по 2009 год)', 'Южный федеральный округ (с 2010 года)', 'Южный федеральный округ (с 29.07.2016)', 'Республика Адыгея (Адыгея)', 'Республика Калмыкия', 'Республика Крым', 'Краснодарский край', 'Астраханская область', 'Волгоградская область', 'Ростовская область', 'Город федерального значения Севастополь', 'Северо-Кавказский федеральный округ', 'Республика Дагестан', 'Республика Ингушетия', 'Кабардино-Балкарская Республика', 'Карачаево-Черкесская Республика', 'Республика Северная Осетия-Алания', 'Чеченская Республика', 'Ставропольский край', 'Приволжский федеральный округ', 'Республика Башкортостан', 'Республика Марий Эл', 'Республика Мордовия', 'Республика Татарстан (Татарстан)', 'Удмуртская Республика', 'Чувашская Республика - Чувашия', 'Пермский край', 'Коми-Пермяцкий округ, входящий в состав Пермского края', 'Кировская область', 'Нижегородская область', 'Оренбургская область', 'Пензенская область', 'Самарская область', 'Саратовская область', 'Ульяновская область', 'Уральский федеральный округ', 'Курганская область', 'Свердловская область', 'Тюменская область', 'Ханты-Мансийский автономный округ - Югра (Тюменская область)', 'Ямало-Ненецкий автономный округ (Тюменская область)', 'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)', 'Челябинская область', 'Сибирский федеральный округ', 'Сибирский федеральный округ (с 03.11.2018)', 'Республика Алтай', 'Республика Тыва', 'Республика Хакасия', 'Алтайский край', 'Красноярский край', 'Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)', 'Эвенкийский автономный округ (Красноярский край)', 'Иркутская область', 'Усть-Ордынский Бурятский округ', 'Кемеровская область - Кузбасс', 'Новосибирская область', 'Омская область', 'Томская область', 'Дальневосточный федеральный округ', 'Дальневосточный федеральный округ (с 03.11.2018)', 'Республика Бурятия', 'Забайкальский край', 'Агинский Бурятский округ (Забайкальский край)', 'Республика Саха (Якутия)', 'Камчатский край', 'Корякский округ, входящий в состав Камчатского края', 'Приморский край', 'Хабаровский край', 'Амурская область', 'Магаданская область', 'Сахалинская область', 'Еврейская автономная область', 'Чукотский автономный округ', 'Северный район', 'Северо-Западный район', 'Центральный район', 'Волго-Вятский район', 'Центрально-Черноземный район', 'Поволжский район', 'Северо-Кавказский район', 'Уральский район', 'Западно-Сибирский район', 'Восточно-Сибирский район', 'Дальневосточный район']
- settlements: ["все население", "городское население", "сельское население"]
- genders: ["Мужчины", "Женщины", "Все"]
- years: [1991 - 2025]
- ages: ['Всего', '0', '0-17 лет', '0-4 года', '1 год', '10 лет', '100 и более лет', '10-14 лет', '11 лет', '12 лет', '13 лет', '14 лет', '15 лет', '15-19 лет', '16 лет', '16-29 лет', '17 лет', '18 лет', '19 лет', '2 года', '20 лет', '20-24', '20-24 лет', '20-29 лет', '20-39 лет', '21 год', '22 лет', '23 лет', '24 лет', '25 лет', '25-29', '25-29 лет', '26 лет', '27 лет', '28 лет', '29 лет', '3 года', '30 лет', '30-34', '30-34 лет', '31 лет', '32 лет', '33 лет', '34 лет', '35 лет', '35-39', '35-39 лет', '36 лет', '37 лет', '38 лет', '39 лет', '4 года', '40 лет', '40-44', '40-44 лет', '41 лет', '42 лет', '43 лет', '44 лет', '45 лет', '45-49 лет', '46 лет', '47 лет', '48 лет', '49 лет', '5 лет', '50 лет', '50-54 лет', '51 лет', '52 лет', '53 года', '54 лет', '55 лет', '55-59 лет', '56 лет', '57 лет', '58 лет', '59 лет', '5-9 лет', '6 лет', '60 лет', '60-64 лет', '61 лет', '62 года', '63 года', '64 года', '65 года', '65-69 лет', '66 года', '67 лет', '68 лет', '69 лет', '7 лет', '70 лет', '70-74 лет', '71 год', '72 года', '73 года', '74 года', '75 лет', '75-79 лет', '76 лет', '77 лет', '78 лет', '79 лет', '8 лет', '80 лет', '80-84 года', '81 год', '82 года', '83 года', '84 года', '85 лет', '85-89 лет', '86 лет', '87 лет', '88 лет', '89 лет', '9 лет', '90 лет', '90-94 года', '91 год', '92 года', '93 года', '94 года', '95 лет', '95-99 лет', '96 лет', '97 лет', '98 лет', '99 лет', 'Всего в трудоспособном возрасте (для мужчин 16-59,для женщин 16-54)', 'Моложе трудоспособного', 'Население пенсионного возраста (66 лет и старше)', 'Старше трудоспособного']

Example input: "Выведи данные по численности населения мужчин 0-5 лет в ДФО за 1990-1991"
Example output:
{
  "region": ["Дальневосточный федеральный округ"],
  "age": ["0 лет", "1 год", "2 года", "3 года", "4 года", "5 лет"],
  "settlement": ["все население"],
  "years": [1990, 1991],
  "gender": ["Мужчины"]
}

"""

In [170]:
from mistralai  import Mistral
client = Mistral(api_key = api_key)

with open("mistral_key.txt", "r") as f:
    api_key = f.read()

In [171]:
def create_input_dict(query: str) -> dict:
    dict = {}
    response = client.chat.complete(
        model = "mistral-large-latest",
        messages = [
            {"role" : "system", "content" : system_prompt},
            {"role" : "user", "content" : query}
        ],
        response_format={"type": "json_object"},
        temperature = 0.7,
        n = 1
    )
    return response

In [172]:
query = "Выведи численность мужского населения с 2021 по 2025 по всем регионам"
res = create_input_dict(query)

In [173]:
for choice in res.choices:
    print(choice)

index=0 message=AssistantMessage(content='{\n  "region": ["Российская Федерация", "Центральный федеральный округ", "Белгородская область", "Брянская область", "Владимирская область", "Воронежская область", "Ивановская область", "Калужская область", "Костромская область", "Курская область", "Липецкая область", "Московская область", "Орловская область", "Рязанская область", "Смоленская область", "Тамбовская область", "Тверская область", "Тульская область", "Ярославская область", "Город Москва столица Российской Федерации город федерального значения", "Северо-Западный федеральный округ", "Республика Карелия", "Республика Коми", "Архангельская область", "Ненецкий автономный округ (Архангельская область)", "Архангельская область (кроме Ненецкого автономного округа)", "Вологодская область", "Калининградская область", "Ленинградская область", "Мурманская область", "Новгородская область", "Псковская область", "Город Санкт-Петербург город федерального значения", "Южный федеральный округ (по 200

# Число родившихся по возрасту матери

In [3]:
born = FedStatIndicator(indicator_id = "59992")

In [37]:
born.filter_codes

{'3': 'Год',
 '58335': 'Возраст',
 '57831': 'Классификатор объектов административно-территориального деления (ОКАТО)',
 '30611': 'Единица измерения',
 '33560': 'Период',
 '60363': 'Порядок рождения'}

In [32]:
filter_ids = born.get_filter_values()
filter_ids

['3_2018',
 '3_2019',
 '3_2020',
 '3_2021',
 '58335_1709566',
 '58335_1785567',
 '58335_1785522',
 '58335_1785573',
 '58335_1785526',
 '58335_1785559',
 '58335_1785489',
 '58335_1785494',
 '58335_1785487',
 '58335_1785500',
 '58335_1785491',
 '58335_1785497',
 '58335_1785490',
 '58335_1785502',
 '58335_1785492',
 '58335_1785499',
 '58335_1785488',
 '58335_1785498',
 '58335_1785486',
 '58335_1785501',
 '58335_1785553',
 '58335_1785503',
 '58335_1785551',
 '58335_1785509',
 '58335_1785549',
 '58335_1785507',
 '58335_1785550',
 '58335_1785505',
 '58335_1785554',
 '58335_1785548',
 '58335_1785504',
 '58335_1785552',
 '58335_1785508',
 '58335_1785546',
 '58335_1785506',
 '58335_1785496',
 '58335_1785511',
 '58335_1785495',
 '58335_1785575',
 '58335_1785512',
 '58335_1785560',
 '58335_1785513',
 '58335_1785574',
 '58335_1785547',
 '58335_1785510',
 '58335_1785555',
 '58335_1785515',
 '58335_1785568',
 '58335_1785569',
 '58335_1785514',
 '58335_1710041',
 '58335_1830364',
 '57831_1688487',
 '

In [63]:
data = {
    "lineObjectIds": ["57831", "30611", "58335", "60363"],
    "columnObjectIds": ["30611", "33560", "3"],
    "selectedFilterIds": [
    filter_ids
    ]
}
params = {
    "format" : "excel",
    "id" : 59992
}
try: 
    response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
    response.raise_for_status()
    if "excel" in response.headers.get("Content-Type"):
        df= pd.read_excel(BytesIO(response.content), engine = "xlrd", header = 4) 
    else:
        raise ValueError(
            f"Ответ не содержит excel файла"
        )
except requests.RequestException as e:
    raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")


In [60]:
fil_vals = born.filter_codes
fil_vals

{'3': 'Год',
 '58335': 'Возраст',
 '57831': 'Классификатор объектов административно-территориального деления (ОКАТО)',
 '30611': 'Единица измерения',
 '33560': 'Период',
 '60363': 'Порядок рождения'}

In [65]:
data.get("lineObjectIds")

['57831', '30611', '58335', '60363']